In [2]:

from gensim.models import word2vec
from gensim.corpora import Dictionary, MmCorpus
import pattern.fr
import pattern.en
import pandas as pd
import re

reviews=pd.read_csv('reviews.csv', encoding = 'utf-8')
reviews_fr = reviews[reviews['language'] == 'fr']
reviews_en = reviews[reviews['language'] == 'en']

def tokenize_line(line,lang):
    
    res = line.lower()
    #res = remove_accents(res)
    res = (re.compile(r"[^a-z 'éèëêà#]")).sub(' ',res)
    
    if lang == "fr":
        tok = pattern.fr.parse(res, tokenize = True, lemmata=True).split()
        
    if lang == "en":
        tok = pattern.en.parse(res, tokenize = True, lemmata=True).split()
    
    tokens = [item[4] for sublist in tok for item in sublist if item[1] not in ['.',',',':',';','-',"'",'"','IN','CC','PRP', 'DT','RB',')','(']]
    
    return tokens 

In [3]:
tokenize_line('testé. deux.trois','fr')

[u'test\xe9', u'deux', u'trois']

In [4]:
reviews_en['tokens'] = reviews_en['reviewDescription'].apply(lambda x: tokenize_line(x,'en'))

/home/ubuntu/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [5]:
sentences = reviews_en.tokens.tolist()
model = word2vec.Word2Vec(sentences, size=20, window=5, min_count=2, workers=4)

In [24]:
#sentences
model.most_similar('compensate')
#model.save('reviews_model')

[(u'maternity', 0.9756664037704468),
 (u'labor', 0.9628701210021973),
 (u'con', 0.9607158303260803),
 (u'vacation', 0.9581053853034973),
 (u'parental', 0.9564428329467773),
 (u'worth', 0.954730749130249),
 (u'alright', 0.9514485001564026),
 (u'wage', 0.9507989287376404),
 (u'ot', 0.9507003426551819),
 (u'bit', 0.9477230310440063)]

# visualization

In [1]:
words = [u'management',u'job',u'work',u'salary',u'remuneration',u'culture',u'mindset',u'digital',u'crm',u'ethic',u'balance',u'experience']

for w in words:
    words = words + ([u[0] for u in model.most_similar(w,topn=10)])


NameError: name 'model' is not defined

In [274]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

embedding = np.array([model[x] for x in words])

pca = PCA(n_components=2)
pca.fit(embedding)
embedding_pca = np.transpose(pca.transform(embedding))
embedding_pca.shape

(2L, 72L)

In [275]:
plt.figure(figsize=(50, 50))
plt.scatter(embedding_pca[0], embedding_pca[1])

for index,(x,y) in enumerate(np.transpose(embedding_pca)):
    plt.text(x,y,words[index])
plt.show()


In [278]:
([u[0] for u in model.most_similar(w,topn=10)]).tolist()

AttributeError: 'list' object has no attribute 'tolist'